In [5]:
# Experiment 2
# Define activation functions to search
# Importing necessary libraries
import kagglehub
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.regularizers import l1, l2

#Required for the notebook's reproduciiility
np.random.seed(2)
tf.random.set_seed(2)

# Download latest version
path = kagglehub.dataset_download("muhammadtahir194/movies-dataset-tmdb-top-rated")

print("Path to dataset files:", path)
file_path=""
for file in os.listdir(path):
    if file.endswith(".csv"):
        file_path = os.path.join(path, file)
        break

df = pd.read_csv(file_path)
print(df.head())
df.dropna(inplace=True)
df['is_popular'] = df['popularity'] > 4.0
df['is_popular'] = df['is_popular'].astype(int) #form of one-hot encodinng

# Convert the release_date column to datetime format and extracting only the year
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year.astype(int)
df.drop(columns=['release_date'], inplace=True)

df.drop(columns=['title', 'overview'], inplace=True)
print(df.head())
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X, y = df.drop(columns=['id','popularity', 'is_popular']), df['is_popular']
print("X shape:", X.shape)
print("y shape:", y.shape)

stratify_val = y if len(np.unique(y)) < 20 else None
print("Unique y values in the dataset: ", len(np.unique(y)))

# Splitting data into training (70%), validation (15%), and test (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=2, stratify=stratify_val)
stratify_val_temp = y_temp if stratify_val is not None else None
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=2, stratify=stratify_val_temp)

print(f"Training Set: {X_train.shape}, Validation Set: {X_val.shape}, Test Set: {X_test.shape}")

# Scaling the numerical features
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_val[numerical_cols] = scaler.transform(X_val[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])



ACTIVATION_FUNCTIONS = ['relu', 'tanh', 'sigmoid', 'elu']


def build_and_train_model_different_activations(neurons=32, layers=2, activation_list=None, dropout_rate=0.3,
                                                optimizer='adam',
                                                learning_rate=0.001, batch_size=16, regularization=None,
                                                batch_norm=False):
    # If no activation list is provided, we will define default to 'relu' for all layers
    if activation_list is None:
        activation_list = ['relu'] * layers

    model = Sequential()

    # Input Layer
    model.add(Dense(neurons, activation=activation_list[0], input_shape=(X_train.shape[1],),
                    kernel_regularizer=regularization))

    if batch_norm:
        model.add(BatchNormalization())

    model.add(Dropout(dropout_rate))

    # Hidden Layers
    for i in range(1, layers):
        model.add(Dense(neurons, activation=activation_list[i], kernel_regularizer=regularization))
        if batch_norm:
            model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))

    # Output Layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile Model
    if optimizer == "adam":
        optimizer = keras.optimizers.Adam(learning_rate)
    elif optimizer == "rmsprop":
        optimizer = keras.optimizers.RMSprop(learning_rate)
    else:
        optimizer = keras.optimizers.SGD(learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train Model
    history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_data=(X_val, y_val), verbose=0)

    # Evaluate Performance
    val_acc = max(history.history['val_accuracy'])
    return model, val_acc, activation_list, history


import itertools

# Global best accuracy and best config
best_overall_acc = 0
best_overall_config = {}


Path to dataset files: /root/.cache/kagglehub/datasets/muhammadtahir194/movies-dataset-tmdb-top-rated/versions/1
        id                     title  \
0      278  The Shawshank Redemption   
1      238             The Godfather   
2      240     The Godfather Part II   
3      424          Schindler's List   
4  1356039             Counterattack   

                                            overview release_date  popularity  \
0  Imprisoned in the 1940s for the double murder ...   1994-09-23       5.522   
1  Spanning the years 1945 to 1955, a chronicle o...   1972-03-14       5.317   
2  In the continuing saga of the Corleone crime f...   1974-12-20       4.747   
3  The true story of how businessman Oskar Schind...   1993-12-15       4.457   
4  When a hostage rescue mission creates a new en...   2025-02-27       9.430   

   vote_average  vote_count  
0         8.708       27883  
1         8.689       21151  
2         8.570       12771  
3         8.567       16219  
4        

In [2]:
#Experiment 2 - different activation functions
ACTIVATION_FUNCTIONS = ['relu', 'tanh', 'sigmoid', 'elu']

def build_and_train_model_different_activations(neurons=32, layers=2, activation_list=None, dropout_rate=0.3, optimizer='adam',
                          learning_rate=0.001, batch_size=16, regularization=None, batch_norm=False):

    # If no activation list is provided, we will define default to 'relu' for all layers
    if activation_list is None:
        activation_list = ['relu'] * layers

    model = Sequential()

    # Input Layer
    model.add(Dense(neurons, activation=activation_list[0], input_shape=(X_train.shape[1],),
                    kernel_regularizer=regularization))

    if batch_norm:
        model.add(BatchNormalization())

    model.add(Dropout(dropout_rate))

    # Hidden Layers
    for i in range(1, layers):
        model.add(Dense(neurons, activation=activation_list[i], kernel_regularizer=regularization))
        if batch_norm:
            model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))

    # Output Layer
    model.add(Dense(1, activation='sigmoid'))

    # Compile Model
    if optimizer == "adam":
        optimizer = keras.optimizers.Adam(learning_rate)
    elif optimizer == "rmsprop":
        optimizer = keras.optimizers.RMSprop(learning_rate)
    else:
        optimizer = keras.optimizers.SGD(learning_rate)

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train Model
    history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_data=(X_val, y_val), verbose=0)

    # Evaluate Performance
    val_acc = max(history.history['val_accuracy'])
    return model, val_acc, activation_list, history



In [3]:


import itertools

# Global best accuracy and best config
best_overall_acc = 0
best_overall_config = {}


def search_best_activation(neurons=32, layers=2, dropout_rate=0.3, optimizer='adam',
                           learning_rate=0.001, batch_size=16, regularization=None, batch_norm=False):
    """
    Searches for the best activation function layout for a given neuron-layer configuration.
    """
    global best_overall_acc, best_overall_config  # Ensure global best tracking

    best_acc_for_config = 0
    best_model = None
    best_activation = None
    best_params_for_config = {}

    # Generate all possible activation combinations
    activation_combinations = list(itertools.product(ACTIVATION_FUNCTIONS, repeat=layers))[:20] #let's limit the amount of possible combinations

    for activation_list in activation_combinations:
        print(f"Testing activation functions: {activation_list} for Neurons={neurons}, Layers={layers}")

        model, val_acc, activation_used, _ = build_and_train_model_different_activations(
            neurons=neurons, layers=layers, activation_list=list(activation_list),
            dropout_rate=dropout_rate, optimizer=optimizer, learning_rate=learning_rate,
            batch_size=batch_size, regularization=regularization, batch_norm=batch_norm
        )

        # Update best accuracy for this neuron-layer configuration
        if val_acc > best_acc_for_config:
            best_acc_for_config = val_acc
            best_model = model
            best_activation = activation_used
            best_params_for_config = {
                'neurons': neurons,
                'layers': layers,
                'activation_list': best_activation,
                'dropout_rate': dropout_rate,
                'optimizer': optimizer,
                'learning_rate': learning_rate,
                'batch_size': batch_size,
                'regularization': regularization,
                'batch_norm': batch_norm
            }

    print(
        f"Best Activation Set for Neurons={neurons}, Layers={layers}: {best_activation} -> Accuracy: {best_acc_for_config}")

    # Update overall best configuration
    if best_acc_for_config > best_overall_acc:
        best_overall_acc = best_acc_for_config
        best_overall_config = best_params_for_config
        print(f"New Overall Best Accuracy: {best_overall_acc} with Configuration: {best_overall_config}")

    return best_model, best_activation, best_acc_for_config, best_params_for_config


# Search for the best activation functions for different layers and neuron configurations
layer = 4
neurons = [4, 8, 16, 32]

# Predefined best hyperparameters
params = {
    'dropout_rate': 0.3,
    'optimizer': 'adam',
    'learning_rate': 0.001,
    'batch_size': 16,
    'regularization': None,
    'batch_norm': False
}

# Store results for each configuration
results = []
for neuron in neurons:
    model, activation, val_acc, best_config = search_best_activation(neurons=neuron, layers=layer, **params)
    results.append((f'Neurons={neuron}, Layers={layer}', {
        'params': best_config,
        'activation': activation,
        'val_acc': val_acc
    }))

# Print final best overall configuration
print(f"\nBest Overall Configuration:\n{best_overall_config} -> Accuracy: {best_overall_acc}")

Testing activation functions: ('relu', 'relu', 'relu', 'relu') for Neurons=4, Layers=4


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Testing activation functions: ('relu', 'relu', 'relu', 'tanh') for Neurons=4, Layers=4


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Testing activation functions: ('relu', 'relu', 'relu', 'sigmoid') for Neurons=4, Layers=4


...

Testing activation functions: ('relu', 'tanh', 'relu', 'sigmoid') for Neurons=32, Layers=4


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Testing activation functions: ('relu', 'tanh', 'relu', 'elu') for Neurons=32, Layers=4


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best Activation Set for Neurons=32, Layers=4: ['relu', 'relu', 'relu', 'relu'] -> Accuracy: 0.8964174389839172
New Overall Best Accuracy: 0.8964174389839172 with Configuration: {'neurons': 32, 'layers': 4, 'activation_list': ['relu', 'relu', 'relu', 'relu'], 'dropout_rate': 0.3, 'optimizer': 'adam', 'learning_rate': 0.001, 'batch_size': 16, 'regularization': None, 'batch_norm': False}

Best Overall Configuration:
{'neurons': 32, 'layers': 4, 'activation_list': ['relu', 'relu', 'relu', 'relu'], 'dropout_rate': 0.3, 'optimizer': 'adam', 'learning_rate': 0.001, 'batch_size': 16, 'regularization': None, 'batch_norm': False} -> Accuracy: 0.8964174389839172


In [ ]:


import itertools

# Global best accuracy and best config
best_overall_acc = 0
best_overall_config = {}


def search_best_activation(neurons=32, layers=2, dropout_rate=0.3, optimizer='adam',
                           learning_rate=0.001, batch_size=16, regularization=None, batch_norm=False):
    """
    Searches for the best activation function layout for a given neuron-layer configuration.
    """
    global best_overall_acc, best_overall_config  # Ensure global best tracking

    best_acc_for_config = 0
    best_model = None
    best_activation = None
    best_params_for_config = {}

    # Generate all possible activation combinations
    activation_combinations = list(itertools.product(ACTIVATION_FUNCTIONS, repeat=layers))[:20] #let's limit the amount of possible combinations

    for activation_list in activation_combinations:
        print(f"Testing activation functions: {activation_list} for Neurons={neurons}, Layers={layers}")

        model, val_acc, activation_used, _ = build_and_train_model_different_activations(
            neurons=neurons, layers=layers, activation_list=list(activation_list),
            dropout_rate=dropout_rate, optimizer=optimizer, learning_rate=learning_rate,
            batch_size=batch_size, regularization=regularization, batch_norm=batch_norm
        )

        # Update best accuracy for this neuron-layer configuration
        if val_acc > best_acc_for_config:
            best_acc_for_config = val_acc
            best_model = model
            best_activation = activation_used
            best_params_for_config = {
                'neurons': neurons,
                'layers': layers,
                'activation_list': best_activation,
                'dropout_rate': dropout_rate,
                'optimizer': optimizer,
                'learning_rate': learning_rate,
                'batch_size': batch_size,
                'regularization': regularization,
                'batch_norm': batch_norm
            }

    print(
        f"Best Activation Set for Neurons={neurons}, Layers={layers}: {best_activation} -> Accuracy: {best_acc_for_config}")

    # Update overall best configuration
    if best_acc_for_config > best_overall_acc:
        best_overall_acc = best_acc_for_config
        best_overall_config = best_params_for_config
        print(f"New Overall Best Accuracy: {best_overall_acc} with Configuration: {best_overall_config}")

    return best_model, best_activation, best_acc_for_config, best_params_for_config


# Search for the best activation functions for different layers and neuron configurations
layer = 4
neurons = [ 64, 128, 512]

# Predefined best hyperparameters
params = {
    'dropout_rate': 0.3,
    'optimizer': 'adam',
    'learning_rate': 0.001,
    'batch_size': 16,
    'regularization': None,
    'batch_norm': False
}

# Store results for each configuration
results = []
for neuron in neurons:
    model, activation, val_acc, best_config = search_best_activation(neurons=neuron, layers=layer, **params)
    results.append((f'Neurons={neuron}, Layers={layer}', {
        'params': best_config,
        'activation': activation,
        'val_acc': val_acc
    }))

# Print final best overall configuration
print(f"\nBest Overall Configuration:\n{best_overall_config} -> Accuracy: {best_overall_acc}")

Testing activation functions: ('relu', 'relu', 'relu', 'relu') for Neurons=64, Layers=4


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Testing activation functions: ('relu', 'relu', 'relu', 'tanh') for Neurons=64, Layers=4


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Testing activation functions: ('relu', 'relu', 'relu', 'sigmoid') for Neurons=64, Layers=4


...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Testing activation functions: ('relu', 'relu', 'elu', 'elu') for Neurons=64, Layers=4


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Testing activation functions: ('relu', 'tanh', 'relu', 'relu') for Neurons=64, Layers=4


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
